# Tarea 2 AID

- Bruno Benkel - 201204111-2
- Felipe Chacon - 201303017-3
- Diego Wilhelm - 201303059-9

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import cluster
from sklearn import metrics
from statsmodels.sandbox.tools.tools_pca import pcasvd
from sklearn.neighbors import kneighbors_graph # Técnica de k-Vecino mas cercano
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.cross_validation import train_test_split 
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from scipy import stats, integrate
%matplotlib inline

# Hepatitis

 1. Class: DIE, LIVE
 2. AGE: 10, 20, 30, 40, 50, 60, 70, 80
 3. SEX: male, female
 4. STEROID: no, yes
 5. ANTIVIRALS: no, yes
 6. FATIGUE: no, yes
 7. MALAISE: no, yes
 8. ANOREXIA: no, yes
 9. LIVER BIG: no, yes
 10. LIVER FIRM: no, yes
 11. SPLEEN PALPABLE: no, yes
 12. SPIDERS: no, yes
 13. ASCITES: no, yes
 14. VARICES: no, yes
 15. BILIRUBIN: 0.39, 0.80, 1.20, 2.00, 3.00, 4.00
 16. ALK PHOSPHATE: 33, 80, 120, 160, 200, 250
 17. SGOT: 13, 100, 200, 300, 400, 500, 
 18. ALBUMIN: 2.1, 3.0, 3.8, 4.5, 5.0, 6.0
 19. PROTIME: 10, 20, 30, 40, 50, 60, 70, 80, 90
 20. HISTOLOGY: no, yes

In [4]:
df1 = pd.read_csv('data/hepatitis.data')

In [5]:
#Quitar las filas que tengan missing values '?'
index = []
for i in range(df1.shape[0]):
    if('?' in df1.iloc[i].values):
        index.append(i)
df1 = df1.drop(index)        

In [6]:
#Sacar la columna Class, que es la que se intenta predecir
target1 = df1['Class']
df1 = df1.drop('Class',axis=1)

In [7]:
mat = df1.as_matrix()

## Clustering

Fueron utilizados K-Means, Average Linkage y Mean Shift.

### K-Means

In [8]:
kmeans = cluster.KMeans(n_clusters=2)
kmeans.fit(mat)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [9]:
print("Silhouette Coefficient: %0.6f"
      % metrics.silhouette_score(mat, kmeans.labels_))

Silhouette Coefficient: 0.482762


### Average Linkage

In [10]:
algoritmo = cluster.AgglomerativeClustering(linkage="average", 
                                            affinity="cityblock", n_clusters=2)
# Linkage: complete, average, ward
# Affinity: “euclidean”, “l1”, “l2”, “manhattan”, "cityblock", “cosine”, o ‘precomputed’
algoritmo.fit(mat)

AgglomerativeClustering(affinity='cityblock', compute_full_tree='auto',
            connectivity=None, linkage='average', memory=None,
            n_clusters=2, pooling_func=<function mean at 0x7f7e6407f400>)

In [11]:
print("Silhouette Coefficient: %0.6f"
      % metrics.silhouette_score(mat, algoritmo.labels_))

Silhouette Coefficient: 0.671057


### Mean Shift

In [12]:
ancho_banda = cluster.estimate_bandwidth(mat, quantile=0.15)
mean_shift_alg = cluster.MeanShift(bandwidth=ancho_banda, bin_seeding=True)
mean_shift_alg.fit(mat)

MeanShift(bandwidth=59.752845675029263, bin_seeding=True, cluster_all=True,
     min_bin_freq=1, n_jobs=1, seeds=None)

In [13]:
print("Silhouette Coefficient: %0.6f"
      % metrics.silhouette_score(mat, mean_shift_alg.labels_))

Silhouette Coefficient: 0.368808


## Clasificadores

In [14]:
H_entrenamiento, H_test, target1_entrenamiento, target1_test = train_test_split(df1, target1, test_size = 0.5, random_state=0)

sc1 = StandardScaler()
sc1.fit(H_entrenamiento) 
Z1_entrenamiento = sc1.transform(H_entrenamiento)
Z1_test = sc1.transform(H_test)

clasificador1 = LinearDiscriminantAnalysis()
clasificador1.fit(Z1_entrenamiento, target1_entrenamiento)
target1_pred1 = clasificador1.predict(Z1_test)
accuracy1 = accuracy_score(target1_test, target1_pred1)
precision1 = precision_score(target1_test, target1_pred1, average="macro")
F1 = f1_score(target1_test, target1_pred1, average='macro')

clasificador2 = QuadraticDiscriminantAnalysis()
clasificador2.fit(Z1_entrenamiento, target1_entrenamiento)
target1_pred2 = clasificador2.predict(Z1_test)
accuracy2 = accuracy_score(target1_test, target1_pred2)
precision2 = precision_score(target1_test, target1_pred2, average="macro")
F2 = f1_score(target1_test, target1_pred2, average='macro')

clasificador3=SVC(kernel="linear", C=0.025)
clasificador3.fit(Z1_entrenamiento, target1_entrenamiento)
target1_pred3 = clasificador3.predict(Z1_test)
accuracy3 = accuracy_score(target1_test, target1_pred3)
precision3 = precision_score(target1_test, target1_pred3, average="macro")
F3 = f1_score(target1_test, target1_pred3, average='macro')

print("[Clasificador 1] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy1, precision1, F1))
print("[Clasificador 2] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy2, precision2, F2))
print("[Clasificador 3] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy3, precision3, F3))

[Clasificador 1] Accuracy: 0.750  -  Precision: 0.60  -  F Score: 0.609
[Clasificador 2] Accuracy: 0.850  -  Precision: 0.42  -  F Score: 0.459
[Clasificador 3] Accuracy: 0.875  -  Precision: 0.76  -  F Score: 0.736


### KFold

In [15]:
scores1 = cross_val_score(clasificador1, df1, target1, cv=10, scoring='accuracy')
scores2 = cross_val_score(clasificador2, df1, target1, cv=10, scoring='accuracy')
scores3 = cross_val_score(clasificador3, df1, target1, cv=10, scoring='accuracy')

print("[Clasificador1] Mean: %0.3f - Standard Deviation: %0.3f"%(scores1.mean(), scores1.std()))
print("[Clasificador2] Mean: %0.3f - Standard Deviation: %0.3f"%(scores2.mean(), scores2.std()))
print("[Clasificador3] Mean: %0.3f - Standard Deviation: %0.3f"%(scores3.mean(), scores3.std()))

[Clasificador1] Mean: 0.761 - Standard Deviation: 0.181
[Clasificador2] Mean: 0.840 - Standard Deviation: 0.042
[Clasificador3] Mean: 0.814 - Standard Deviation: 0.057


## PCA

In [16]:
df1 = df1.astype(float)
df1 = (df1 - df1.mean()) / df1.std()
pca1 = pcasvd(df1, keepdim=15, demean=False)

In [17]:
H_entrenamiento, H_test, target1_entrenamiento, target1_test = train_test_split(pca1[0], target1, test_size = 0.5, random_state=0)

sc1 = StandardScaler()
sc1.fit(H_entrenamiento) 
Z1_entrenamiento = sc1.transform(H_entrenamiento)
Z1_test = sc1.transform(H_test)

clasificador1 = LinearDiscriminantAnalysis()
clasificador1.fit(Z1_entrenamiento, target1_entrenamiento)
target1_pred1 = clasificador1.predict(Z1_test)
accuracy1 = accuracy_score(target1_test, target1_pred1)
precision1 = precision_score(target1_test, target1_pred1, average="macro")
F1 = f1_score(target1_test, target1_pred1, average='macro')

clasificador2 = QuadraticDiscriminantAnalysis()
clasificador2.fit(Z1_entrenamiento, target1_entrenamiento)
target1_pred2 = clasificador2.predict(Z1_test)
accuracy2 = accuracy_score(target1_test, target1_pred2)
precision2 = precision_score(target1_test, target1_pred2, average="macro")
F2 = f1_score(target1_test, target1_pred2, average='macro')

clasificador3=SVC(kernel="linear", C=0.025)
clasificador3.fit(Z1_entrenamiento, target1_entrenamiento)
target1_pred3 = clasificador3.predict(Z1_test)
accuracy3 = accuracy_score(target1_test, target1_pred3)
precision3 = precision_score(target1_test, target1_pred3, average="macro")
F3 = f1_score(target1_test, target1_pred3, average='macro')

print("[Clasificador 1] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy1, precision1, F1))
print("[Clasificador 2] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy2, precision2, F2))
print("[Clasificador 3] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy3, precision3, F3))

[Clasificador 1] Accuracy: 0.675  -  Precision: 0.56  -  F Score: 0.551
[Clasificador 2] Accuracy: 0.850  -  Precision: 0.42  -  F Score: 0.459
[Clasificador 3] Accuracy: 0.800  -  Precision: 0.56  -  F Score: 0.543


In [18]:
scores1 = cross_val_score(clasificador1, df1, target1, cv=10, scoring='accuracy')
scores2 = cross_val_score(clasificador2, df1, target1, cv=10, scoring='accuracy')
scores3 = cross_val_score(clasificador3, df1, target1, cv=10, scoring='accuracy')

print("[Clasificador1] Mean: %0.3f - Standard Deviation: %0.3f"%(scores1.mean(), scores1.std()))
print("[Clasificador2] Mean: %0.3f - Standard Deviation: %0.3f"%(scores2.mean(), scores2.std()))
print("[Clasificador3] Mean: %0.3f - Standard Deviation: %0.3f"%(scores3.mean(), scores3.std()))

[Clasificador1] Mean: 0.761 - Standard Deviation: 0.181
[Clasificador2] Mean: 0.840 - Standard Deviation: 0.042
[Clasificador3] Mean: 0.837 - Standard Deviation: 0.079


# Pima Indian Diabetes

1. Number of times pregnant
2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test
3. Diastolic blood pressure (mm Hg)
4. Triceps skin fold thickness (mm)
5. 2-Hour serum insulin (mu U/ml)
6. Body mass index (weight in kg/(height in m)^2)
7. Diabetes pedigree function
8. Age (years)
9. Class variable (0 or 1)

In [19]:
df2 = pd.read_csv('data/diabetes.data')

In [20]:
#Quitar las filas que tengan missing values '?'
index = []
for i in range(df2.shape[0]):
    if('?' in df2.iloc[i].values):
        index.append(i)
df2 = df2.drop(index)  

In [21]:
target2 = df2['Class']
df2 = df2.drop('Class',axis=1)
mat = df2.as_matrix()

## Clustering

### K-Means

In [22]:
kmeans = cluster.KMeans(n_clusters=2)
kmeans.fit(mat)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [23]:
print("Silhouette Coefficient: %0.6f"
      % metrics.silhouette_score(mat, kmeans.labels_))

Silhouette Coefficient: 0.568790


### Average Linkage

In [24]:
algoritmo = cluster.AgglomerativeClustering(linkage="average", 
                                            affinity="cityblock", n_clusters=2)
# Linkage: complete, average, ward
# Affinity: “euclidean”, “l1”, “l2”, “manhattan”, "cityblock", “cosine”, o ‘precomputed’
algoritmo.fit(mat)

AgglomerativeClustering(affinity='cityblock', compute_full_tree='auto',
            connectivity=None, linkage='average', memory=None,
            n_clusters=2, pooling_func=<function mean at 0x7f7e6407f400>)

In [25]:
print("Silhouette Coefficient: %0.6f"
      % metrics.silhouette_score(mat, algoritmo.labels_))

Silhouette Coefficient: 0.728518


### Mean-Shift

In [26]:
ancho_banda = cluster.estimate_bandwidth(mat, quantile=0.15)
mean_shift_alg = cluster.MeanShift(bandwidth=ancho_banda, bin_seeding=True)
mean_shift_alg.fit(mat)

MeanShift(bandwidth=63.883927652192064, bin_seeding=True, cluster_all=True,
     min_bin_freq=1, n_jobs=1, seeds=None)

In [27]:
print("Silhouette Coefficient: %0.6f"
      % metrics.silhouette_score(mat, mean_shift_alg.labels_))

Silhouette Coefficient: 0.428159


## Clasificación

In [28]:
D_entrenamiento, D_test, target2_entrenamiento, target2_test = train_test_split(df2, target2, test_size = 0.5, random_state=0)

sc2 = StandardScaler()
sc2.fit(D_entrenamiento) 
Z2_entrenamiento = sc2.transform(D_entrenamiento)
Z2_test = sc2.transform(D_test)

clasificador1 = LinearDiscriminantAnalysis()
clasificador1.fit(Z2_entrenamiento, target2_entrenamiento)
target2_pred1 = clasificador1.predict(Z2_test)
accuracy1 = accuracy_score(target2_test, target2_pred1)
precision1 = precision_score(target2_test, target2_pred1, average="macro")
F1 = f1_score(target2_test, target2_pred1, average='macro')

clasificador2 = QuadraticDiscriminantAnalysis()
clasificador2.fit(Z2_entrenamiento, target2_entrenamiento)
target2_pred2 = clasificador2.predict(Z2_test)
accuracy2 = accuracy_score(target2_test, target2_pred2)
precision2 = precision_score(target2_test, target2_pred2, average="macro")
F2 = f1_score(target2_test, target2_pred2, average='macro')

clasificador3=SVC(kernel="linear", C=0.025)
clasificador3.fit(Z2_entrenamiento, target2_entrenamiento)
target2_pred3 = clasificador3.predict(Z2_test)
accuracy3 = accuracy_score(target2_test, target2_pred3)
precision3 = precision_score(target2_test, target2_pred3, average="macro")
F3 = f1_score(target2_test, target2_pred3, average='macro')

print("[Clasificador 1] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy1, precision1, F1))
print("[Clasificador 2] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy2, precision2, F2))
print("[Clasificador 3] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy3, precision3, F3))

[Clasificador 1] Accuracy: 0.781  -  Precision: 0.77  -  F Score: 0.741
[Clasificador 2] Accuracy: 0.766  -  Precision: 0.74  -  F Score: 0.728
[Clasificador 3] Accuracy: 0.789  -  Precision: 0.79  -  F Score: 0.743


In [29]:
#KFold
scores1 = cross_val_score(clasificador1, df2, target2, cv=10, scoring='accuracy')
scores2 = cross_val_score(clasificador2, df2, target2, cv=10, scoring='accuracy')
scores3 = cross_val_score(clasificador3, df2, target2, cv=10, scoring='accuracy')

print("[Clasificador1] Mean: %0.3f - Standard Deviation: %0.3f"%(scores1.mean(), scores1.std()))
print("[Clasificador2] Mean: %0.3f - Standard Deviation: %0.3f"%(scores2.mean(), scores2.std()))
print("[Clasificador3] Mean: %0.3f - Standard Deviation: %0.3f"%(scores3.mean(), scores3.std()))

[Clasificador1] Mean: 0.773 - Standard Deviation: 0.035
[Clasificador2] Mean: 0.740 - Standard Deviation: 0.028
[Clasificador3] Mean: 0.766 - Standard Deviation: 0.025


## PCA

In [30]:
df2 = df2.astype(float)
df2 = (df2 - df2.mean()) / df2.std()
pca2 = pcasvd(df2, keepdim=6, demean=False)

In [31]:
D_entrenamiento, D_test, target2_entrenamiento, target2_test = train_test_split(pca2[0], target2, test_size = 0.5, random_state=0)

sc2 = StandardScaler()
sc2.fit(D_entrenamiento) 
Z2_entrenamiento = sc2.transform(D_entrenamiento)
Z2_test = sc2.transform(D_test)

clasificador1 = LinearDiscriminantAnalysis()
clasificador1.fit(Z2_entrenamiento, target2_entrenamiento)
target2_pred1 = clasificador1.predict(Z2_test)
accuracy1 = accuracy_score(target2_test, target2_pred1)
precision1 = precision_score(target2_test, target2_pred1, average="macro")
F1 = f1_score(target2_test, target2_pred1, average='macro')

clasificador2 = QuadraticDiscriminantAnalysis()
clasificador2.fit(Z2_entrenamiento, target2_entrenamiento)
target2_pred2 = clasificador2.predict(Z2_test)
accuracy2 = accuracy_score(target2_test, target2_pred2)
precision2 = precision_score(target2_test, target2_pred2, average="macro")
F2 = f1_score(target2_test, target2_pred2, average='macro')

clasificador3=SVC(kernel="linear", C=0.025)
clasificador3.fit(Z2_entrenamiento, target2_entrenamiento)
target2_pred3 = clasificador3.predict(Z2_test)
accuracy3 = accuracy_score(target2_test, target2_pred3)
precision3 = precision_score(target2_test, target2_pred3, average="macro")
F3 = f1_score(target2_test, target2_pred3, average='macro')

print("[Clasificador 1] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy1, precision1, F1))
print("[Clasificador 2] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy2, precision2, F2))
print("[Clasificador 3] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy3, precision3, F3))

[Clasificador 1] Accuracy: 0.781  -  Precision: 0.77  -  F Score: 0.741
[Clasificador 2] Accuracy: 0.688  -  Precision: 0.64  -  F Score: 0.626
[Clasificador 3] Accuracy: 0.781  -  Precision: 0.77  -  F Score: 0.738


In [32]:
#KFold
scores1 = cross_val_score(clasificador1, df2, target2, cv=10, scoring='accuracy')
scores2 = cross_val_score(clasificador2, df2, target2, cv=10, scoring='accuracy')
scores3 = cross_val_score(clasificador3, df2, target2, cv=10, scoring='accuracy')

print("[Clasificador1] Mean: %0.3f - Standard Deviation: %0.3f"%(scores1.mean(), scores1.std()))
print("[Clasificador2] Mean: %0.3f - Standard Deviation: %0.3f"%(scores2.mean(), scores2.std()))
print("[Clasificador3] Mean: %0.3f - Standard Deviation: %0.3f"%(scores3.mean(), scores3.std()))

[Clasificador1] Mean: 0.773 - Standard Deviation: 0.035
[Clasificador2] Mean: 0.740 - Standard Deviation: 0.028
[Clasificador3] Mean: 0.771 - Standard Deviation: 0.033


# Water Treatment Plant

|N. | Identificador | Descripción |
| -- |:---:|:---|
| 1 | Q-E       | (input flow to plant)   |
| 2 | ZN-E      | (input Zinc to plant) |
| 3 | PH-E      | (input pH to plant)  |
| 4 | DBO-E     | (input Biological demand of oxygen to plant)  |
| 5 | DQO-E     | (input chemical demand of oxygen to plant) |
| 6 | SS-E      | (input suspended solids to plant)   |
| 7 | SSV-E     | (input volatile supended solids to plant) |
| 8 | SED-E     | (input sediments to plant)  |
| 9 | COND-E    | (input conductivity to plant)  |
|10 | PH-P      | (input pH to primary settler) |
|11 | DBO-P     | (input Biological demand of oxygen to primary settler) |
|12 | SS-P      | (input suspended solids to primary settler) |
|13 | SSV-P     | (input volatile supended solids to primary settler) |
|14 | SED-P     | (input sediments to primary settler)  |
|15 | COND-P    | (input conductivity to primary settler) |
|16 | PH-D      | (input pH to secondary settler)  |
|17 | DBO-D     | (input Biological demand of oxygen to secondary settler) |
|18 | DQO-D     | (input chemical demand of oxygen to secondary settler) |
|19 | SS-D      | (input suspended solids to secondary settler) |
|20 | SSV-D     | (input volatile supended solids to secondary settler) |
|21 | SED-D     | (input sediments to secondary settler)   |
|22 | COND-D    | (input conductivity to secondary settler)  |
|23 | PH-S      | (output pH)    |
|24 | DBO-S     | (output Biological demand of oxygen) |
|25 | DQO-S     | (output chemical demand of oxygen) |
|26 | SS-S      | (output suspended solids) |
|27 | SSV-S     | (output volatile supended solids)  |
|28 | SED-S     | (output sediments)  |
|29 | COND-S    | (output conductivity) |
|30 | RD-DBO-P  | (performance input Biological demand of oxygen in primary settler) |
|31 | RD-SS-P   | (performance input suspended solids to primary settler) |
|32 | RD-SED-P  | (performance input sediments to primary settler) |
|33 | RD-DBO-S  | (performance input Biological demand of oxygen to secondary settler) |
|34 | RD-DQO-S  | (performance input chemical demand of oxygen to secondary settler) |
|35 | RD-DBO-G  | (global performance input Biological demand of oxygen) |
|36 | RD-DQO-G  | (global performance input chemical demand of oxygen) |
|37 | RD-SS-G   | (global performance input suspended solids)  |
|38 | RD-SED-G  | (global performance input sediments) |

Hay 13 clases a las que puede pertenecer esto

* Class 1: Normal situation
* Class 2: Secondary settler problems-1
* Class 3: Secondary settler problems-2
* Class 4: Secondary settler problems-3
* Class 5: Normal situation with performance over the mean
* Class 6: Solids overload-1
* Class 7: Secondary settler problems-4
* Class 8: Storm-1
* Class 9: Normal situation with a low influent
* Class 10: Storm-2
* Class 11: Normal situation
* Class 12: Storm-3
* Class 13: Solids overload-2

In [166]:
class1 = "D-1/3/90 to D-12/3/90, D-16/3/90 to D-30/3/90, D-1/2/90 to D-19/2/90, D-21/2/90 to D-28/2/90, D-1/1/90 to D-26/1/90, \
D-29/1/90 to D-31/1/90, D-1/6/90 to D-4/6/90, D-6/6/90 to D-8/6/90, D-24/6/90, D-25/6/90, D-28/6/90, D-29/6/90, D-1/5/90 to D-6/5/90, D-8/5/90 to D-20/5/90, \
D-24/5/90, D-25/5/90, D-29/5/90, D-1/4/90, D-4/4/90 to D-8/4/90, D-10/4/90 to D-20/4/90, \
D-27/4/90, D-2/7/90, D-4/7/90 to D-8/7/90, D-12/7/90 to D-15/7/90, D-19/7/90, D-23/7/90, \
D-26/7/90, D-4/9/90, D-5/9/90, D-23/9/90, D-28/9/90, D-30/9/90, D-17/8/90, D-21/8/90 to D-25/8/90, \
D-29/8/90, D-30/8/90, D-3/12/90, D-9/12/90, D-16/12/90 to D-20/12/90, D-23/12/90, D-24/12/90, \
D-27/12/90 to D-30/12/90, D-6/11/90 to D-8/11/90, D-14/11/90, D-16/11/90, D-18/11/90, \
D-20/11/90, D-21/11/90, D-27/11/90, D-10/10/90, D-18/10/90, D-29/10/90, D-30/10/90, \
D-3/3/91 to D-6/3/91, D-10/3/91 to D-12/3/91, D-18/3/91, D-20/3/91, D-27/3/91, D-29/3/91, \
D-3/2/91, D-5/2/91, D-8/2/91, D-14/2/91, D-17/2/91, D-18/2/91, D-21/2/91 to D-24/2/91, \
D-1/1/91, D-2/1/91, D-6/1/91, D-8/1/91, D-10/1/91 to D-20/1/91, D-25/1/91, D-2/5/91, D-3/5/91, \
D-7/5/91, D-14/5/91, D-15/5/91, D-17/5/91, D-19/5/91, D-21/5/91 to D-23/5/91, D-1/4/91 to D-3/4/91, \
D-5/4/91 to D-12/4/91, D-15/4/91 to D-21/4/91, D-23/4/91, D-1/7/91, D-3/7/91, D-4/7/91, D-7/7/91, \
D-10/7/91 to D-12/7/91, D-15/7/91, D-16/7/91, D-22/7/91 to D-25/7/91, D-28/7/91, D-30/7/91, D-31/7/91, \
D-2/6/91 to D-4/6/91, D-6/6/91, D-7/6/91, D-13/6/91, D-16/6/91 to D-21/6/91, D-25/6/91 to D-30/6/91, \
D-4/10/91, D-6/10/91, D-17/10/91 to D-30/10/91, D-1/8/91, D-2/8/91, D-27/8/91, D-29/8/91"

class5 = "D-28/1/90, D-10/6/90 to D-22/6/90, D-26/6/90, D-27/6/90, D-7/5/90, D-21/5/90 to D-23/5/90, \
D-27/5/90, D-28/5/90, D-30/5/90, D-2/4/90, D-3/4/90, D-9/4/90, D-22/4/90 to D-26/4/90, D-1/7/90, \
D-3/7/90, D-9/7/90 to D-11/7/90, D-16/7/90 to D-18/7/90, D-20/7/90, D-22/7/90, D-24/7/90, D-25/7/90, \
D-27/7/90 to D-31/7/90, D-2/9/90, D-3/9/90, D-6/9/90 to D-13/9/90, D-16/9/90 to D-21/9/90, \
D-24/9/90 to D-27/9/90, D-1/8/90 to D-7/8/90, D-16/8/90, D-28/8/90, D-31/8/90, D-7/12/90, \
D-2/11/90, D-5/11/90, D-9/11/90, D-12/11/90, D-13/11/90, D-1/10/90 to D-5/10/90, D-24/10/90, \
D-25/10/90, D-1/3/91, D-8/3/91, D-17/3/91, D-26/3/91, D-31/3/91, D-9/1/91, D-10/5/91, D-16/5/91, \
D-20/5/91, D-29/5/91, D-30/5/91, D-14/4/91, D-22/4/91, D-24/4/91, D-25/4/91, D-5/7/91, D-8/7/91, \
D-9/7/91, D-21/7/91, D-26/7/91, D-5/6/91, D-10/6/91, D-12/6/91, D-14/6/91, D-2/10/91, D-8/10/91, \
D-9/10/91, D-11/10/91, D-13/10/91, D-16/10/91"

#Esta mal escrito en el .names del dataset, especificamente el extracto "D-4/8/91  D-9/8/91 to D-26/8/91"
class9 = "D-8/8/90 to D-10/8/90, D-13/8/90, D-15/8/90, D-19/8/90, D-20/8/90, D-27/8/90, D-1/11/90, \
D-4/11/90, D-11/11/90, D-19/11/90, D-7/10/90 to D-9/10/90, D-12/10/90 to D-17/10/90, \
D-21/10/90, D-23/10/90, D-26/10/90, D-28/10/90, D-7/3/91, D-24/3/91, D-25/3/91, \
D-1/5/91, D-5/5/91, D-8/5/91, D-9/5/91, D-12/5/91, D-13/5/91, D-26/5/91, D-27/5/91, \
D-26/4/91, D-28/4/91, D-29/4/91, D-2/7/91, D-14/7/91, D-29/7/91, D-9/6/91, D-24/6/91, \
D-1/10/91, D-3/10/91, D-5/10/91, D-12/10/91, D-15/10/91, D-4/8/91 to D-26/8/91, \
D-28/8/91, D-30/8/91"

class11 = "D-2/12/90, D-4/12/90, D-6/12/90, D-10/12/90 to D-14/12/90, D-21/12/90, D-26/12/90, \
D-15/11/90, D-22/11/90 to D-26/11/90, D-28/11/90 to D-30/11/90, D-19/10/90, \
D-13/3/91 to D-15/3/91, D-19/3/91, D-21/3/91, D-22/3/91, D-1/2/91, D-4/2/91, \
D-6/2/91, D-7/2/91, D-10/2/91 to  D-13/2/91, D-15/2/91, D-19/2/91, \
D-25/2/91 to D-28/2/91, D-3/1/91, D-4/1/91, D-7/1/91, D-21/1/91 to D-24/1/91, \
D-27/1/91 to D-31/1/91, D-6/5/91, D-4/4/91"

In [127]:
def addToClass(classN, Class, n):
    for date in classN.split(", "):
        if "to" in date:
            dates = date.split(" to ")
            month = dates[0].split("/")[1]
            year = dates[0].split("/")[2]
            bot = int(dates[0].split("/")[0].split("-")[1])
            top = int(dates[1].split("/")[0].split("-")[1])
            for i in range(bot,top+1):
                Class["D-" + str(i) + "/" + month + "/" + year] = n
        else:
            Class[date] = n

In [167]:
Class = dict()

#Clase 1
addToClass(class1, Class, 1)

#Clase 2 a 4        
Class["D-13/3/90"] = 2
Class["D-14/3/90"] = 3
Class["D-15/3/90"] = 4
Class["D-17/3/90"] = 4
Class["D-18/7/91"] = 4
Class["D-19/7/91"] = 4

#Clase 5
addToClass(class5, Class, 5)

#Clase 6 a 8
Class["D-5/6/90"] = 6
Class["D-28/5/91"] = 6
Class["D-31/5/91"] = 6
Class["D-29/4/90"] = 7
Class["D-14/9/90"] = 8

#Clase 9
addToClass(class9, Class, 9)

#Clase 10
Class["D-12/8/90"] = 10

#Clase 11
addToClass(class11, Class, 11)

#Clase 12 y 13
Class["D-22/10/90"] = 12
Class["D-24/5/90"] = 13

In [168]:
df3 = pd.read_csv('data/water-treatment.data')

In [169]:
#Quitar las filas que tengan missing values '?'
index = []
for j in df3.index:
    if('?' in df3.loc[j].values):
        index.append(j)
df3 = df3.drop(index)      

In [170]:
df3["Class"] = df3.index

In [171]:
for j in df3.index:
    df3["Class"][j] = Class[j]

In [173]:
df3

,Q-E,ZN-E,PH-E,DBO-E,DQO-E,SS-E,SSV-E,SED-E,COND-E,PH-P,...,RD-DBO-P,RD-SS-P,RD-SED-P,RD-DBO-S,RD-DQO-S,RD-DBO-G,RD-DQO-G,RD-SS-G,RD-SED-G,Class
D-5/3/90,35023,3.50,7.9,205,588,192,65.6,4.5,2430,7.8,...,33.1,64.2,95.3,87.3,72.3,90.2,82.3,89.6,100,1
D-11/3/90,29156,2.50,7.7,206,451,194,69.1,4.5,1249,7.7,...,46.1,43.6,92.5,85.6,58.2,92.2,73.8,90.2,99.4,1
D-12/3/90,39246,2.00,7.8,172,506,200,69.0,5.0,1865,7.8,...,21.2,59.7,90.8,88.4,66.1,89.0,69.0,86.5,99.6,1
D-13/3/90,42393,0.70,7.9,189,478,230,67.0,5.5,1410,8.1,...,0.6,45.8,92.0,11.6,25.7,19.6,36.0,43.0,36.4,2
D-18/3/90,40923,3.50,7.6,146,329,188,57.4,2.5,1300,7.6,...,32.7,33.3,90.0,82.6,61.3,87.0,71.4,78.2,99.2,1
D-19/3/90,43830,1.5,7.8,177,512,214,58.9,5.5,1605,7.7,...,28.0,72.7,92.7,78.8,36.6,85.9,60.4,90.7,100,1
D-20/3/90,39165,1.20,7.4,250,447,252,61.1,7.0,1533,7.4,...,49.8,58.3,92.3,89.9,96.8,94.4,98.1,92.1,100,1
D-22/3/90,37419,1.20,7.6,219,446,222,61.3,5.5,1600,7.7,...,35.3,57.5,92.0,75.6,49.3,80.8,60.8,76.1,99.6,1
D-23/3/90,40983,3.00,7.6,182,431,214,57.0,7.0,1591,7.5,...,20.1,64.5,93.6,86.9,68.1,87.4,72.2,88.3,100,1
D-25/3/90,42217,8.50,7.5,138,333,240,55.0,3.8,1087,7.5,...,29.4,55.4,91.3,85.2,68.0,88.4,81.4,92.9,100,1


In [35]:
np.random.seed(1)
target3 = np.random.randint(2,13,size=380)
#dataset no incluye el atributo de la clase
mat = df3.as_matrix()

## Clustering

### K-Means

In [36]:
kmeans = cluster.KMeans(n_clusters=13)
kmeans.fit(mat)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=13, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [37]:
print("Silhouette Coefficient: %0.6f"
      % metrics.silhouette_score(mat, kmeans.labels_))

Silhouette Coefficient: 0.381638


### Average Linkage

In [38]:
algoritmo = cluster.AgglomerativeClustering(linkage="average", 
                                            affinity="cityblock", n_clusters=13)
# Linkage: complete, average, ward
# Affinity: “euclidean”, “l1”, “l2”, “manhattan”, "cityblock", “cosine”, o ‘precomputed’
algoritmo.fit(mat)

AgglomerativeClustering(affinity='cityblock', compute_full_tree='auto',
            connectivity=None, linkage='average', memory=None,
            n_clusters=13, pooling_func=<function mean at 0x7f7e6407f400>)

In [39]:
print("Silhouette Coefficient: %0.6f"
      % metrics.silhouette_score(mat, algoritmo.labels_))

Silhouette Coefficient: 0.342311


### Mean-Shift

In [40]:
ancho_banda = cluster.estimate_bandwidth(mat, quantile=0.15)
mean_shift_alg = cluster.MeanShift(bandwidth=ancho_banda, bin_seeding=True)
mean_shift_alg.fit(mat)

MeanShift(bandwidth=2703.5498315775735, bin_seeding=True, cluster_all=True,
     min_bin_freq=1, n_jobs=1, seeds=None)

In [41]:
print("Silhouette Coefficient: %0.6f"
      % metrics.silhouette_score(mat, mean_shift_alg.labels_))

Silhouette Coefficient: 0.512379


## Clasificación

In [42]:
W_entrenamiento, W_test, target3_entrenamiento, target3_test = train_test_split(df3, target3, test_size = 0.5, random_state=0)

sc3 = StandardScaler()
sc3.fit(W_entrenamiento) 
Z3_entrenamiento = sc3.transform(W_entrenamiento)
Z3_test = sc3.transform(W_test)

clasificador1 = LinearDiscriminantAnalysis()
clasificador1.fit(Z3_entrenamiento, target3_entrenamiento)
target3_pred1 = clasificador1.predict(Z3_test)
accuracy1 = accuracy_score(target3_test, target3_pred1)
precision1 = precision_score(target3_test, target3_pred1, average="macro")
F1 = f1_score(target3_test, target3_pred1, average='macro')

clasificador2 = QuadraticDiscriminantAnalysis()
clasificador2.fit(Z3_entrenamiento, target3_entrenamiento)
target3_pred2 = clasificador2.predict(Z3_test)
accuracy2 = accuracy_score(target3_test, target3_pred2)
precision2 = precision_score(target3_test, target3_pred2, average="macro")
F2 = f1_score(target3_test, target3_pred2, average='macro')

clasificador3=SVC(kernel="linear", C=0.025)
clasificador3.fit(Z3_entrenamiento, target3_entrenamiento)
target3_pred3 = clasificador3.predict(Z3_test)
accuracy3 = accuracy_score(target3_test, target3_pred3)
precision3 = precision_score(target3_test, target3_pred3, average="macro")
F3 = f1_score(target3_test, target3_pred3, average='macro')

print("[Clasificador 1] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy1, precision1, F1))
print("[Clasificador 2] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy2, precision2, F2))
print("[Clasificador 3] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy3, precision3, F3))

[Clasificador 1] Accuracy: 0.116  -  Precision: 0.10  -  F Score: 0.103
[Clasificador 2] Accuracy: 0.095  -  Precision: 0.11  -  F Score: 0.095
[Clasificador 3] Accuracy: 0.084  -  Precision: 0.11  -  F Score: 0.078


In [43]:
#KFold
scores1 = cross_val_score(clasificador1, df3, target3, cv=10, scoring='accuracy')
scores2 = cross_val_score(clasificador2, df3, target3, cv=10, scoring='accuracy')
scores3 = cross_val_score(clasificador3, df3, target3, cv=10, scoring='accuracy')

print("[Clasificador1] Mean: %0.3f - Standard Deviation: %0.3f"%(scores1.mean(), scores1.std()))
print("[Clasificador2] Mean: %0.3f - Standard Deviation: %0.3f"%(scores2.mean(), scores2.std()))
print("[Clasificador3] Mean: %0.3f - Standard Deviation: %0.3f"%(scores3.mean(), scores3.std()))

KeyboardInterrupt: 

## PCA

In [ ]:
df3 = df3.astype(float)
df3 = (df3 - df3.mean()) / df3.std()
pca3 = pcasvd(df3, keepdim=30, demean=False)

In [ ]:
W_entrenamiento, W_test, target3_entrenamiento, target3_test = train_test_split(pca3[0], target3, test_size = 0.5, random_state=0)

sc3 = StandardScaler()
sc3.fit(W_entrenamiento) 
Z3_entrenamiento = sc3.transform(W_entrenamiento)
Z3_test = sc3.transform(W_test)

clasificador1 = LinearDiscriminantAnalysis()
clasificador1.fit(Z3_entrenamiento, target3_entrenamiento)
target3_pred1 = clasificador1.predict(Z3_test)
accuracy1 = accuracy_score(target3_test, target3_pred1)
precision1 = precision_score(target3_test, target3_pred1, average="macro")
F1 = f1_score(target3_test, target3_pred1, average='macro')

clasificador2 = QuadraticDiscriminantAnalysis()
clasificador2.fit(Z3_entrenamiento, target3_entrenamiento)
target3_pred2 = clasificador2.predict(Z3_test)
accuracy2 = accuracy_score(target3_test, target3_pred2)
precision2 = precision_score(target3_test, target3_pred2, average="macro")
F2 = f1_score(target3_test, target3_pred2, average='macro')

clasificador3=SVC(kernel="linear", C=0.025)
clasificador3.fit(Z3_entrenamiento, target3_entrenamiento)
target3_pred3 = clasificador3.predict(Z3_test)
accuracy3 = accuracy_score(target3_test, target3_pred3)
precision3 = precision_score(target3_test, target3_pred3, average="macro")
F3 = f1_score(target3_test, target3_pred3, average='macro')

print("[Clasificador 1] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy1, precision1, F1))
print("[Clasificador 2] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy2, precision2, F2))
print("[Clasificador 3] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy3, precision3, F3))

In [ ]:
#KFold
scores1 = cross_val_score(clasificador1, df3, target3, cv=10, scoring='accuracy')
scores2 = cross_val_score(clasificador2, df3, target3, cv=10, scoring='accuracy')
scores3 = cross_val_score(clasificador3, df3, target3, cv=10, scoring='accuracy')

print("[Clasificador1] Mean: %0.3f - Standard Deviation: %0.3f"%(scores1.mean(), scores1.std()))
print("[Clasificador2] Mean: %0.3f - Standard Deviation: %0.3f"%(scores2.mean(), scores2.std()))
print("[Clasificador3] Mean: %0.3f - Standard Deviation: %0.3f"%(scores3.mean(), scores3.std()))